In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

# from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import numpy as np
import time

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from scipy.linalg import sqrtm

In [ ]:
!pip install -q git+https://github.com/tensorflow/examples.git

In [ ]:
from tensorflow_examples.models.pix2pix import pix2pix

In [ ]:
# Attempting to connect to a TPU cluster and configurating it
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    # strategy = tf.distribute.experimental.TPUStrategy(tpu)

except:
    print("Connection failed.")
    strategy = tf.distribute.get_strategy()
  
print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE
    
print(tf.__version__)

Connection failed.
Number of replicas: 1
2.3.0


In [ ]:
# Setting up the paths to the .tfrec files from the dataset
# MONET_PATH = "./dataset/monet/"
# PHOTO_PATH = "./dataset/photo/"

MONET_TRAIN_PATH = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/Dataset/alt-monet-tfrecord/train/"
MONET_TEST_PATH = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/Dataset/alt-monet-tfrecord/test/"
# MONET_PATH = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/Dataset/monet_tfrec/"
PHOTO_TRAIN_PATH = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/Dataset/alt-photos-tfrecord/train/"
PHOTO_TEST_PATH = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/Dataset/alt-photos-tfrecord/test/"

In [ ]:
# Loading the dataset filenames to use them in our models later on
MONET_TRAIN_FILENAMES = tf.io.gfile.glob(str(MONET_TRAIN_PATH + "*.tfrec"))
print('Monet train TFRecord Files:', len(MONET_TRAIN_FILENAMES))
MONET_TEST_FILENAMES = tf.io.gfile.glob(str(MONET_TEST_PATH + "*.tfrec"))
print('Monet TEST TFRecord Files:', len(MONET_TEST_FILENAMES))

PHOTO_TRAIN_FILENAMES = tf.io.gfile.glob(str(PHOTO_TRAIN_PATH + "*.tfrec"))
print('Photos train TFRecord Files:', len(PHOTO_TRAIN_FILENAMES))
PHOTO_TEST_FILENAMES = tf.io.gfile.glob(str(PHOTO_TEST_PATH + "*.tfrec"))
print('Photos test TFRecord Files:', len(PHOTO_TEST_FILENAMES))

Monet train TFRecord Files: 4
Monet TEST TFRecord Files: 1
Photos train TFRecord Files: 4
Photos test TFRecord Files: 1


In [ ]:
# Auxiliary functions to manage the tfrecords
IMAGE_SIZE = [256, 256]

# Adjusting the image dimensions and scale
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

# Parsing each element of the dataset
def read_tfrecord(example):
    tfrecord_format = {
        # "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        # "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

# Reads the tfrecords and parses them using the read_tfrecord function
def load_dataset(filenames, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord)#, num_parallel_calls=AUTOTUNE)
    return dataset

In [ ]:
monet_train_ds = load_dataset(MONET_TRAIN_FILENAMES, labeled=True).batch(1)
monet_test_ds = load_dataset(MONET_TEST_FILENAMES, labeled=True).batch(1)

photo_train_ds = load_dataset(PHOTO_TRAIN_FILENAMES, labeled=True).batch(1)
photo_test_ds = load_dataset(PHOTO_TEST_FILENAMES, labeled=True).batch(1)

In [ ]:
LAMBDA = 10
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
# Discriminator loss
def discriminator_loss(real, generated):
    # Compares the real image with a matrix of ones, which is the ideal result
    real_loss = loss_obj(tf.ones_like(real), real)

    # Compares the generated image with a matrix of zeros, which is the ideal result 
    generated_loss = loss_obj(tf.zeros_like(generated), generated)

    total_disc_loss = real_loss + generated_loss

    return total_disc_loss * 0.5
  
# Generator loss
def generator_loss(generated):
    # Compares the generated image with a matrix of ones.
    # The goal is to have a generated image as close to a real image as possible, i.e., close to a matrix of ones.
    return loss_obj(tf.ones_like(generated), generated)

# Cycle consistency loss
def calc_cycle_loss(real_image, cycled_image):
    # Mean absolute error between the real image and the cycled image (generated back to the original domain)
    # Example:
    # real_image = X
    # cycled image = F(G(X)) = X_hat ~ X
    loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))

    return LAMBDA * loss1
  
# Identity loss
def identity_loss(real_image, same_image):
    # Mean absolute error between an image and its generator (i.e. a horse image with a horse image generator)
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return LAMBDA * 0.5 * loss

In [ ]:
class CycleGan(keras.Model):
    def __init__(
        self,
        monet_generator,
        photo_generator,
        monet_discriminator,
        photo_discriminator,
    ):
        super(CycleGan, self).__init__()
        self.m_gen = monet_generator
        self.p_gen = photo_generator
        self.m_disc = monet_discriminator
        self.p_disc = photo_discriminator
        
    def compile(
        self,
        m_gen_optimizer,
        p_gen_optimizer,
        m_disc_optimizer,
        p_disc_optimizer,
        gen_loss_fn,
        disc_loss_fn,
        cycle_loss_fn,
        identity_loss_fn
    ):
        super(CycleGan, self).compile()
        self.m_gen_optimizer = m_gen_optimizer
        self.p_gen_optimizer = p_gen_optimizer
        self.m_disc_optimizer = m_disc_optimizer
        self.p_disc_optimizer = p_disc_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn
        self.cycle_loss_fn = cycle_loss_fn
        self.identity_loss_fn = identity_loss_fn
        
    def train_step(self, batch_data):
        real_monet, real_photo = batch_data
        
        with tf.GradientTape(persistent=True) as tape:
            # Generator G translates X -> Y
            # Generator F translates Y -> X.

            fake_monet = self.m_gen(real_photo, training=True)
            cycled_photo = self.p_gen(fake_monet, training=True)

            fake_photo = self.p_gen(real_monet, training=True)
            cycled_monet = self.m_gen(fake_photo, training=True)

            # same_x and same_y are used for identity loss.
            same_monet = self.m_gen(real_monet, training=True)
            same_photo = self.p_gen(real_photo, training=True)

            # discriminator used to check, inputing real images
            disc_real_monet = self.m_disc(real_monet, training=True)
            disc_real_photo = self.p_disc(real_photo, training=True)

            # discriminator used to check, inputing fake images
            disc_fake_monet = self.m_disc(fake_monet, training=True)
            disc_fake_photo = self.p_disc(fake_photo, training=True)

            # evaluates generator loss
            monet_gen_loss = self.gen_loss_fn(disc_fake_monet)
            photo_gen_loss = self.gen_loss_fn(disc_fake_photo)

            # evaluates total cycle consistency loss
            total_cycle_loss = self.cycle_loss_fn(real_monet, cycled_monet) + self.cycle_loss_fn(real_photo, cycled_photo)

            # evaluates total generator loss
            total_monet_gen_loss = monet_gen_loss + total_cycle_loss + self.identity_loss_fn(real_monet, same_monet)
            total_photo_gen_loss = photo_gen_loss + total_cycle_loss + self.identity_loss_fn(real_photo, same_photo)

            # evaluates discriminator loss
            monet_disc_loss = self.disc_loss_fn(disc_real_monet, disc_fake_monet)
            photo_disc_loss = self.disc_loss_fn(disc_real_photo, disc_fake_photo)

        # Calculate the gradients for generator and discriminator
        monet_generator_gradients = tape.gradient(total_monet_gen_loss, self.m_gen.trainable_variables)
        photo_generator_gradients = tape.gradient(total_photo_gen_loss, self.p_gen.trainable_variables)

        monet_discriminator_gradients = tape.gradient(monet_disc_loss, self.m_disc.trainable_variables)
        photo_discriminator_gradients = tape.gradient(photo_disc_loss, self.p_disc.trainable_variables)

        # Apply the gradients to the optimizer
        self.m_gen_optimizer.apply_gradients(zip(monet_generator_gradients, self.m_gen.trainable_variables))
        self.p_gen_optimizer.apply_gradients(zip(photo_generator_gradients, self.p_gen.trainable_variables))
        self.m_disc_optimizer.apply_gradients(zip(monet_discriminator_gradients, self.m_disc.trainable_variables))
        self.p_disc_optimizer.apply_gradients(zip(photo_discriminator_gradients, self.p_disc.trainable_variables))
        
        return {
            "monet_gen_loss": total_monet_gen_loss,
            "photo_gen_loss": total_photo_gen_loss,
            "monet_disc_loss": monet_disc_loss,
            "photo_disc_loss": photo_disc_loss
        }

In [ ]:
# Creating the generators and discriminators for the CycleGAN

OUTPUT_CHANNELS = 3 # RGB channels

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm') # X -> Y
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm') # Y -> X

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False) # Dx
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False) # Dy

# Optimizers
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
with strategy.scope():
    
    # monet_generator, photo_generator, monet_discriminator, photo_discriminator
    cycle_gan_model = CycleGan(
        generator_g, generator_f, discriminator_x, discriminator_y
    )

    cycle_gan_model.compile(
        m_gen_optimizer = generator_g_optimizer,
        p_gen_optimizer = generator_f_optimizer,
        m_disc_optimizer = discriminator_x_optimizer,
        p_disc_optimizer = discriminator_y_optimizer,
        gen_loss_fn = generator_loss,
        disc_loss_fn = discriminator_loss,
        cycle_loss_fn = calc_cycle_loss,
        identity_loss_fn = identity_loss
    )

In [ ]:
import gc

model_inception = InceptionV3(include_top=False, pooling='avg', input_shape= (256,256, 3))

def calculate_fid_precalculated_stats(model, mu_real, cov_real, images_gen):
  act_gen = model.predict(images_gen)

 
  del images_gen
  gc.collect()

  mu_gen, cov_gen = act_gen.mean(axis=0), np.cov(act_gen, rowvar = False)
  
  ssdiff = np.sum((mu_real - mu_gen)**2.0)
  covmean = sqrtm(cov_real.dot(cov_gen))
  if np.iscomplexobj(covmean):
    covmean = covmean.real
   
  fid = ssdiff + np.trace(cov_real + cov_gen - 2.0 * covmean)
  return fid

In [ ]:
monet_ds = list(monet_test_ds.take(-1)) + list(monet_train_ds.take(-1))
monet_real = np.array([(img[0] * 127.5 + 127.5).numpy().astype(np.float32) for img in monet_ds])
display(monet_real.shape)

(1193, 256, 256, 3)

In [ ]:
monet_real = preprocess_input(monet_real)
act_real = model_inception.predict(monet_real)
mu_real, cov_real = act_real.mean(axis=0), np.cov(act_real, rowvar = False)

In [ ]:
import os.path
import gc

photo_ds = list(photo_test_ds.take(-1)) + list(photo_train_ds.take(-1))
fids = []

for i in np.arange(21, 111, 10):
  checkpoint_path = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/checkpoints/" + str(i) + "-120split_trainig.h5" 
  print('Arquivo existe: ', os.path.exists(checkpoint_path))

  try:
      cycle_gan_model.built = True
      cycle_gan_model.load_weights(filepath=checkpoint_path)
      print("Successfully loaded the model " + str(i))

  except:
      print("There is no checkpoint to load")

  pred = []  
  for img in photo_ds:
    img_pred = generator_g(img, training = False)[0].numpy()
    img_pred = (img_pred * 127.5 + 127.5).astype(np.float32)
    pred.append(img_pred)

  monet_gen = np.array([img for img in pred])
  monet_gen = preprocess_input(monet_gen)
  fid_i = calculate_fid_precalculated_stats(model_inception, mu_real, cov_real, monet_gen)

  fids.append([i, fid_i])
  print("Fid " + str(fid_i) + " calculado para " + str(i))

  del pred
  del monet_gen
  gc.collect()




Arquivo existe:  True
Successfully loaded the model 21
Fid 77.58551123790514 calculado para 21
Arquivo existe:  True
Successfully loaded the model 31
Fid 73.39263951408883 calculado para 31
Arquivo existe:  True
Successfully loaded the model 41
Fid 73.42120635529338 calculado para 41
Arquivo existe:  True
Successfully loaded the model 51
Fid 74.58160206903887 calculado para 51
Arquivo existe:  True
Successfully loaded the model 61
Fid 70.23957367814737 calculado para 61
Arquivo existe:  True
Successfully loaded the model 71
Fid 71.38538507926017 calculado para 71
Arquivo existe:  True
Successfully loaded the model 81
Fid 73.03217503427533 calculado para 81
Arquivo existe:  True
Successfully loaded the model 91
Fid 73.20088464776667 calculado para 91
Arquivo existe:  True
Successfully loaded the model 101
Fid 72.45309506646394 calculado para 101


In [ ]:
# Fid 4297.421584347764 calculado para 21
# Fid 4690.8566464837695 calculado para 31
# Fid 7343.937043087424 calculado para 41
checkpoint_path = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/checkpoints/120split_trainig.h5" 
print('Arquivo existe: ', os.path.exists(checkpoint_path))

try:
  cycle_gan_model.built = True
  cycle_gan_model.load_weights(filepath=checkpoint_path)
  print("Successfully loaded the model")

except:
  print("There is no checkpoint to load")

pred = []  
for img in photo_ds:
  img_pred = generator_g(img, training = False)[0].numpy()
  img_pred = (img_pred * 127.5 + 127.5).astype(np.float32)
  pred.append(img_pred)

monet_gen = np.array([img for img in pred])
monet_gen = preprocess_input(monet_gen)
fid_i = calculate_fid_precalculated_stats(model_inception, mu_real, cov_real, monet_gen)

fids.append([120, fid_i])

print(fids)

Arquivo existe:  True
Successfully loaded the model
[[120, 72.45314438930383]]


# 3166 fotos não vistas no treinamento

In [ ]:
# PHOTO_TRAIN_PATH = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/Dataset/alt-photos-tfrecord/train/"
# PHOTO_TEST_PATH = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/Dataset/alt-photos-tfrecord/test/"

# PHOTO_TRAIN_FILENAMES = tf.io.gfile.glob(str(PHOTO_TRAIN_PATH + "*.tfrec"))
# print('Photos train TFRecord Files:', len(PHOTO_TRAIN_FILENAMES))
# PHOTO_TEST_FILENAMES = tf.io.gfile.glob(str(PHOTO_TEST_PATH + "*.tfrec"))
# print('Photos test TFRecord Files:', len(PHOTO_TEST_FILENAMES))

# photo_train_ds = load_dataset(PHOTO_TRAIN_FILENAMES, labeled=True).batch(1)
# photo_test_ds = load_dataset(PHOTO_TEST_FILENAMES, labeled=True).batch(1)
import gc

del monet_ds
del monet_real
del act_real
gc.collect()

PHOTO_PATH = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/Dataset/photo_tfrec/photo_fid/"
PHOTO_FILENAMES = tf.io.gfile.glob(str(PHOTO_PATH + "*.tfrec"))
photo_ds = load_dataset(PHOTO_FILENAMES, labeled=True).batch(1)

photo_ds = list(photo_ds)
photo_real = np.array([(img[0] * 127.5 + 127.5).numpy().astype(np.float32) for img in photo_ds])
print(photo_real.shape)



(3166, 256, 256, 3)


In [ ]:
import os.path
import gc

fids = []

for i in np.arange(21, 111, 10):
  gc.collect()
  checkpoint_path = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/checkpoints/" + str(i) + "-120split_trainig.h5" 
  print('Arquivo existe: ', os.path.exists(checkpoint_path))

  try:
      cycle_gan_model.built = True
      cycle_gan_model.load_weights(filepath=checkpoint_path)
      print("Successfully loaded the model " + str(i))

  except:
      print("There is no checkpoint to load")

  pred = []  
  for img in photo_ds:
    img_pred = generator_g(img, training = False)[0].numpy()
    img_pred = (img_pred * 127.5 + 127.5).astype(np.float32)
    pred.append(img_pred)

  monet_gen = np.array([img for img in pred])
  monet_gen = preprocess_input(monet_gen)
  fid_i = calculate_fid_precalculated_stats(model_inception, mu_real, cov_real, monet_gen)

  fids.append([i, fid_i])
  print("Fid " + str(fid_i) + " calculado para " + str(i))

  del pred
  del monet_gen

Arquivo existe:  True
Successfully loaded the model 21
Fid 77.10029908511594 calculado para 21
Arquivo existe:  True
Successfully loaded the model 31
Fid 72.2697242132921 calculado para 31
Arquivo existe:  True
Successfully loaded the model 41
Fid 73.18617094437968 calculado para 41
Arquivo existe:  True
Successfully loaded the model 51
Fid 72.41483427190339 calculado para 51
Arquivo existe:  True
Successfully loaded the model 61
Fid 68.99867525704182 calculado para 61
Arquivo existe:  True
Successfully loaded the model 71
Fid 68.07579036379151 calculado para 71
Arquivo existe:  True
Successfully loaded the model 81
Fid 70.13721350532902 calculado para 81
Arquivo existe:  True
Successfully loaded the model 91
Fid 69.37113404884661 calculado para 91
Arquivo existe:  True
There is no checkpoint to load
Fid 69.37114365785843 calculado para 101


In [ ]:
checkpoint_path = "/content/drive/Shared drives/Equipe Rocket/8_Sem/Aplic AM/checkpoints/120split_trainig.h5" 
print('Arquivo existe: ', os.path.exists(checkpoint_path))

try:
  cycle_gan_model.built = True
  cycle_gan_model.load_weights(filepath=checkpoint_path)
  print("Successfully loaded the model")

except:
  print("There is no checkpoint to load")

pred = []  
for img in photo_ds:
  img_pred = generator_g(img, training = False)[0].numpy()
  img_pred = (img_pred * 127.5 + 127.5).astype(np.float32)
  pred.append(img_pred)

monet_gen = np.array([img for img in pred])
monet_gen = preprocess_input(monet_gen)
fid_i = calculate_fid_precalculated_stats(model_inception, mu_real, cov_real, monet_gen)

fids.append([120, fid_i])

print(fids)

Arquivo existe:  True
Successfully loaded the model
[[21, 77.10029908511594], [31, 72.2697242132921], [41, 73.18617094437968], [51, 72.41483427190339], [61, 68.99867525704182], [71, 68.07579036379151], [81, 70.13721350532902], [91, 69.37113404884661], [101, 69.37114365785843], [120, 68.75711282309082]]


In [ ]:
import plotly.graph_objects as go
import pandas as pd


fids_x = []
fids_y = []

for i in range(10):
  fids_x.append(fids[i][0])
  fids_y.append(fids[i][1])


layout = dict(xaxis=dict(title = "Epochs"))
fig = go.Figure(go.Scatter(x=fids_x, y=fids_y, mode='lines+markers'))
fig.update_layout(title = "Evolução do FID ao longo da fase de treinamento", xaxis_title="Épocas" , yaxis_title="Valor de FID")#, layout = layout)
fig.show()